In [1]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

In [2]:
spark = sparknlp.start()

:: loading settings :: url = jar:file:/usr/local/lib/python3.8/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ec8423a4-3de1-45d8-91d9-c208baa04b97;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.4.4 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in c

	20 artifacts copied, 0 already retrieved (397123kB/371ms)
22/05/16 03:21:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data = spark.createDataFrame([['Xiaoyin is a good person living in Singapore. Jiajia is also a good person. She lives in China']]).toDF('text')

In [4]:
data.show(truncate=False)

+----------------------------------------------------------------------------------------------+
|text                                                                                          |
+----------------------------------------------------------------------------------------------+
|Xiaoyin is a good person living in Singapore. Jiajia is also a good person. She lives in China|
+----------------------------------------------------------------------------------------------+



In [8]:
document = DocumentAssembler().setInputCol('text').setOutputCol('document').setCleanupMode('shrink')

In [10]:
sentence = SentenceDetector().setInputCols('document').setOutputCol('sentence')

In [11]:
sentence.setExplodeSentences(True)

SentenceDetector_551bb8886d4c

In [12]:
tokenizer = Tokenizer().setInputCols('sentence').setOutputCol('token')

In [13]:
tokenizer.setExceptions(['e-mail'])

Tokenizer_806033acac92

In [14]:
checker = NorvigSweetingModel.pretrained().setInputCols('token').setOutputCol('checked')

spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[ | ]spellcheck_norvig download started this may take some time.
Approximate size to download 4.2 MB
[ / ]Download done! Loading the resource.
[ — ]

[OK!]


In [15]:
embeddings = WordEmbeddingsModel.pretrained().setInputCols(['sentence', 'token']).setOutputCol('embeddings')

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ]glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ / ]Download done! Loading the resource.
[OK!]


In [17]:
ner = NerDLModel.pretrained().setInputCols(['sentence','checked','embeddings']).setOutputCol('ner')

ner_dl download started this may take some time.
Approximate size to download 13.6 MB
[OK!]


In [18]:
converter = NerConverter().setInputCols(['sentence','checked','ner']).setOutputCol('chunk')

In [19]:
from pyspark.ml import Pipeline

In [26]:
pipeline = Pipeline().setStages([document, sentence, tokenizer, checker, embeddings, ner, converter])

In [27]:
model = pipeline.fit(data)

In [29]:
result = model.transform(data)

In [32]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|             checked|          embeddings|                 ner|               chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Xiaoyin is a good...|[{document, 0, 93...|[{document, 0, 44...|[{token, 0, 6, Xi...|[{token, 0, 6, Xi...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 6, Xi...|
|Xiaoyin is a good...|[{document, 0, 93...|[{document, 46, 7...|[{token, 46, 51, ...|[{token, 46, 51, ...|[{word_embeddings...|[{named_entity, 4...|[{chunk, 46, 51, ...|
|Xiaoyin is a good...|[{document, 0, 93...|[{document, 76, 9...|[{token, 76, 78, ...|[{token, 76, 78, ...|[{word_embeddings...|[{named_entity, 7...|[{

In [33]:
result.select('checked.result').show(truncate=False)

+--------------------------------------------------------+
|result                                                  |
+--------------------------------------------------------+
|[Xiaoyin, is, a, good, person, living, in, Singapore, .]|
|[Jiajia, is, also, a, good, person, .]                  |
|[She, lives, in, China]                                 |
+--------------------------------------------------------+



In [34]:
result.select('ner.result').show(truncate=False)

+-----------------------------------+
|result                             |
+-----------------------------------+
|[B-PER, O, O, O, O, O, O, B-LOC, O]|
|[B-PER, O, O, O, O, O, O]          |
|[O, O, O, B-LOC]                   |
+-----------------------------------+



In [35]:
result.select('ner.begin', 'ner.end').show(truncate=False)

+----------------------------------+----------------------------------+
|begin                             |end                               |
+----------------------------------+----------------------------------+
|[0, 8, 11, 13, 18, 25, 32, 35, 44]|[6, 9, 11, 16, 23, 30, 33, 43, 44]|
|[46, 53, 56, 61, 63, 68, 74]      |[51, 54, 59, 61, 66, 73, 74]      |
|[76, 80, 86, 89]                  |[78, 84, 87, 93]                  |
+----------------------------------+----------------------------------+



In [36]:
result.select('chunk.result', 'chunk.begin', 'chunk.end').show(truncate=False)

+--------------------+-------+-------+
|result              |begin  |end    |
+--------------------+-------+-------+
|[Xiaoyin, Singapore]|[0, 35]|[6, 43]|
|[Jiajia]            |[46]   |[51]   |
|[China]             |[89]   |[93]   |
+--------------------+-------+-------+



In [37]:
light = LightPipeline(model)

In [38]:
light.annotate('Jessica is a nice girl living in ChengDu.')

{'chunk': ['Jessica', 'ChengDu'],
 'checked': ['Jessica',
  'is',
  'a',
  'nice',
  'girl',
  'living',
  'in',
  'Chengtu',
  '.'],
 'document': ['Jessica is a nice girl living in ChengDu.'],
 'token': ['Jessica',
  'is',
  'a',
  'nice',
  'girl',
  'living',
  'in',
  'ChengDu',
  '.'],
 'ner': ['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'],
 'embeddings': ['Jessica',
  'is',
  'a',
  'nice',
  'girl',
  'living',
  'in',
  'ChengDu',
  '.'],
 'sentence': ['Jessica is a nice girl living in ChengDu.']}